# (0) Setup

In [5]:
# Colab

# !pip install -U rapidsai
# import cudf


In [6]:
!git clone https://github.com/kkonstantin182/wine-classifier.git
%cd wine-classifier
!pip install -r requirements.txt

Cloning into 'wine-classifier'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 108 (delta 57), reused 54 (delta 18), pack-reused 0
Receiving objects: 100% (108/108), 2.32 MiB | 7.90 MiB/s, done.
Resolving deltas: 100% (57/57), done.
/content/wine-classifier/wine-classifier
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:

# import nltk
# nltk.download()

2023-05-07 14:55:03.457890: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-07 14:55:06.545205: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 28.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')


In [8]:
import sys
from notebooks.path_conf import get_project_root
path_src = get_project_root() / "src"
sys.path.append(str(path_src.resolve()))

from src.text_processing import TextProcessing, Vectorization
from src.dataset import Dataset
from src.constants import SEED


In [43]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from tqdm import tqdm
from bornrule import BornClassifier
import multiprocessing as mp
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize
import numpy as np

In [19]:
nltk.download('punkt')
!python -m spacy download it_core_news_sm

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


2023-05-07 14:57:06.210351: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 71.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')


In [10]:
# data_path = get_project_root() / "data" / "dataset1_proc.csv"

# TARGET_MAP = {
#     'Rosato': 0, 
#     'Frizzante': 1, 
#     'Bianco': 2, 
#     'Rosso': 3
# }

# COLUMNS = {
#     'target': ['type'],
#     'text': ['review', 'winery', 'variety'],
#     'numerical': ['price'],
#     'categorical': ['appellation2']
# }

# ds_obj = Dataset(data_path, target_map=TARGET_MAP, columns_names=COLUMNS)

# train_set, test_set = ds_obj()

# tp_obj_clean = TextProcessing(is_lemmatized=True)

# with mp.Pool(mp.cpu_count()) as pool:
#     train_set['text'] = pool.map(tp_obj_clean, train_set['text'])

# with mp.Pool(mp.cpu_count()) as pool:
#     test_set['text'] = pool.map(tp_obj_clean, test_set['text'])

In [11]:
COLUMNS = {
    'target': ['type'],
    'text': ['review', 'winery', 'variety'],
    'numerical': ['price'],
    'categorical': ['appellation2']
}

In [12]:
tp_obj_clean = TextProcessing(is_lemmatized=True)


In [83]:
train_set = pd.read_csv(get_project_root() / "data" / "dataset1_train_text_cleaned.csv", index_col=False)
test_set = pd.read_csv(get_project_root() / "data" / "dataset1_test_text_cleaned.csv", index_col=False)

In [85]:
train_set.drop(columns=['Unnamed: 0'], inplace=True)
test_set.drop(columns=['Unnamed: 0'], inplace=True)

In [86]:
train_set.to_csv("dataset1_train_text_cleaned_.csv", index=False)
test_set.to_csv("dataset1_test_text_cleaned_.csv", index=False)

In [14]:
print(train_set.shape)
print(test_set.shape)


(12000, 5)
(3000, 5)


# (1) Experiments

In [15]:
X_train, X_test = train_set.drop("target", axis=1), test_set.drop("target", axis=1)
y_train, y_test = train_set['target'], test_set['target']

In [68]:
# Define preprocessor for the pipeline
# tfidf_transformer = TfidfVectorizer(tokenizer=tp_obj_clean.tokenize)
tfidf_transformer = TfidfVectorizer(tokenizer=Vectorization.tokenize_it, smooth_idf=True, sublinear_tf=True)
onehot_transformer = OneHotEncoder()
num_transformer = MinMaxScaler()
# num_transformer = StandardScaler()
preprocessor = ColumnTransformer([
    ('tfidf', tfidf_transformer, 'text'),
    ('onehot', onehot_transformer, COLUMNS['categorical']),
    ('num', num_transformer, COLUMNS['numerical'])
])

In [69]:
# Define pipelines for classifiers
pipelines = [
    # ('Logistic Regression', Pipeline([
    #     ('preprocessor', preprocessor),
    #     ('clf', LogisticRegression(random_state=SEED))
    # ])),

    # ('SVM', Pipeline([
    #     ('preprocessor', preprocessor),
    #     ('clf', SVC(random_state=SEED))
    # ])),

    # ('Random Forest', Pipeline([
    #     ('preprocessor', preprocessor),
    #     ('clf', RandomForestClassifier(random_state=SEED))
    # ])),

    ('Born Rule', Pipeline([
        ('preprocessor', preprocessor),
        ('clf', BornClassifier())
    ]))




]

# Define hyperparameters for grid search
hyperparameters = {
    # 'Logistic Regression': {
    #     'clf__solver': ['saga'],
    #     'clf__penalty': ['l1', 'l2', 'elasticnet', 'none'],
    #     'clf__C': [0.1, 1, 10],
    # },

    # 'SVM': {
    #     'clf__C': [0.01, 0.1, 1, 10, 100],
    #     'clf__kernel': ['linear', 'rbf']
    # },
    
    # 'Random Forest': {
    #     'clf__n_estimators': [10, 100, 1000],
    #     'clf__max_depth': [10, 100, None],
    #     'clf__bootstrap': [True, False],
    #     'clf__min_samples_split': [2, 10, 100],
    # },

    'Born Rule': {
        # 'clf__a': [0.25, 0.5, 1.0, 4.0], # Cannot be 0
        # 'clf__b': [.0, 0.25, 0.5, 1.0, 4.0],
        # 'clf__h': [.0, 0.25, 0.5, 1.0, 4.0],
        'clf__a': [1.0, 4.0], # Cannot be 0
        'clf__b': [.0, 0.25]
    }
}

In [70]:
# Define table to store results
results_table = pd.DataFrame(columns=[
    'Classifier', 
    'Hyperparameters', 
    'Accuracy', 
    'Precision', 
    'Recall', 
    'F1-score'])

# Train and evaluate models
for clf_name, pipeline in tqdm(pipelines, desc="Classifiers", total=len(pipelines)):
    print("Training", clf_name)
    clf = pipeline.named_steps['clf']
    hyperparams = hyperparameters[clf_name]
    rs = RandomizedSearchCV(pipeline, hyperparams, cv=5, scoring='accuracy', n_jobs=-1)
    rs.fit(X_train, y_train)
    
    # Make predictions on train data
    y_pred = rs.predict(X_train)
    
    # Compute metrics
    accuracy = accuracy_score(y_train, 
                              y_pred)
    precision,recall,fscore,support=precision_recall_fscore_support(y_train, 
                                                                    y_pred,
                                                                    average='macro')

    # Store results in table
    results_table = results_table.append({
        'Classifier': clf_name,
        'Hyperparameters': rs.best_params_,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-score': fscore
    }, ignore_index=True)
    
    # Print progress bar
    remaining_iters = len(pipelines) - (pipelines.index((clf_name, pipeline)) + 1)
    print(f"{remaining_iters} iterations left")
    print("---------------------------------------------------------")

    # print(report)
    
# Print final results table
print("\nResults table:")
print(results_table)
results_table.to_csv('class_cls_hp_results.csv')

Classifiers:   0%|          | 0/1 [00:00<?, ?it/s]

Training Born Rule


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
<ipython-input-70-8c955ad94686>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_table = results_table.append({
Classifiers: 100%|██████████| 1/1 [01:13<00:00, 73.16s/it]

0 iterations left
---------------------------------------------------------

Results table:
  Classifier                  Hyperparameters  Accuracy  Precision    Recall  \
0  Born Rule  {'clf__b': 0.25, 'clf__a': 1.0}  0.931333   0.962847  0.799334   

   F1-score  
0  0.853029  


In [71]:
results_table 

,Classifier,Hyperparameters,Accuracy,Precision,Recall,F1-score
0,Born Rule,"{'clf__b': 0.25, 'clf__a': 1.0}",0.931333,0.962847,0.799334,0.853029


In [72]:
# Set hyperparameters for Born Rule model
pipelines[0][1].set_params(clf__a=1.0, clf__b=0.25)

# Fit the pipeline
pipelines[0][1].fit(X_train, y_train)

# Make predictions on test data
y_pred = pipelines[0][1].predict(X_test)

# Compute accuracy
accuracy = accuracy_score(y_test, y_pred)

# Print accuracy
print("Accuracy:", accuracy)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


ValueError: ignored

In [77]:
X_test[X_test['Unnamed: 0'] < 0].sum()

Unnamed: 0      0.0
price           0.0
appellation2    0.0
text            0.0
dtype: float64

In [38]:
# Transform the test set using the preprocessor
X_test_transformed = preprocessor.transform(X_test)

# Print the first 5 rows of the transformed test set
print(X_test_transformed[:5])

  (0, 64)	0.21293130797718396
  (0, 76)	0.3443807818150129
  (0, 190)	0.1843159664496377
  (0, 225)	0.0863686849171465
  (0, 667)	0.0781289294377636
  (0, 850)	0.14318258431440348
  (0, 1047)	0.14806283527319983
  (0, 1090)	0.18051394802578702
  (0, 1531)	0.240182241213645
  (0, 1715)	0.12364449767345938
  (0, 2289)	0.2575234698212577
  (0, 2540)	0.16090674787876422
  (0, 2562)	0.1604637780405265
  (0, 2600)	0.2052270116292693
  (0, 2890)	0.11566256842140825
  (0, 2975)	0.16135394459227514
  (0, 3040)	0.12891396114852327
  (0, 3802)	0.1506719984094871
  (0, 4132)	0.2040949309816282
  (0, 4282)	0.11686432298403969
  (0, 4639)	0.12423096053202957
  (0, 4774)	0.07916761450276653
  (0, 4912)	0.05130259340777492
  (0, 6046)	0.10546650864890877
  (0, 6198)	0.13826612224555848
  :	:
  (4, 4629)	0.06897095253972549
  (4, 4655)	0.20050202850640644
  (4, 4703)	0.09298936389808374
  (4, 4720)	0.07567790093003789
  (4, 4729)	0.1201235280642503
  (4, 4774)	0.05013882556236384
  (4, 4912)	0.03249121

In [56]:
hui = pd.DataFrame(X_test_transformed.toarray())

In [57]:
# Assuming `df` is your pandas dataframe
if (hui < 0).any(axis=None):
    print("There are negative values in the dataframe")
else:
    print("There are no negative values in the dataframe")

There are negative values in the dataframe


In [58]:
# find the negative elements
neg_rows, neg_cols = np.where(hui < 0)

In [59]:
print(neg_rows, neg_cols)

[2050] [7794]


In [60]:
hui.iloc[2050, 7794]

-0.0007446016381236044